In [1]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance
!pip install --upgrade nbformat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 KB 2.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 KB 2.8 MB/s eta 0:00:00
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.7.1
    Uninstalling nbformat-5.7.1:
      Successfully uninstalled nbformat-5.7.1


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

REST API Call

In this lab, we will be using the <a href=https://www.coingecko.com/en/api?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork1005-2022-01-01>CoinGecko API to create one of these candlestick graphs for DogeCoin. We will use the API to get the price data for 90 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 30 candlesticks and use that to generate the candlestick graph. Although we are using the CoinGecko API we will use a Python client/wrapper for the API called PyCoinGecko. PyCoinGecko will make performing the requests easy and it will deal with the enpoint targeting.

Lets start off by getting the data we need. Using the get_coin_market_chart_by_id(id, vs_currency, days). id is the name of the coin you want, vs_currency is the currency you want the price in, and days is how many days back from today you want.

In [3]:
cg = CoinGeckoAPI()

dogecoin_data = cg.get_coin_market_chart_by_id(id='dogecoin', vs_currency='usd', days=90)

In [4]:
type(dogecoin_data)

dict

The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.

In [7]:
dogecoin_price_data = dogecoin_data['prices']

dogecoin_price_data[0:5]

[[1667552467923, 0.12126359297828024],
 [1667556132395, 0.11895915104138488],
 [1667559673857, 0.11914794890706493],
 [1667563311239, 0.11942590764011511],
 [1667566959234, 0.1215390223475979]]

Finally lets turn this data into a Pandas DataFrame.

In [8]:
data = pd.DataFrame(dogecoin_price_data, columns=['TimeStamp', 'Price'])

Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called Date. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime.

In [12]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))
data.head

<bound method NDFrame.head of           TimeStamp     Price        date
0     1667552467923  0.121264  2022-11-04
1     1667556132395  0.118959  2022-11-04
2     1667559673857  0.119148  2022-11-04
3     1667563311239  0.119426  2022-11-04
4     1667566959234  0.121539  2022-11-04
...             ...       ...         ...
2164  1675317795793  0.094378  2023-02-02
2165  1675318027322  0.094295  2023-02-02
2166  1675321429244  0.093629  2023-02-02
2167  1675324842040  0.093329  2023-02-02
2168  1675326089000  0.093695  2023-02-02

[2169 rows x 3 columns]>

Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

In [13]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

Finally we are now ready to use plotly to create our Candlestick Chart.

In [14]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()